In [ ]:
import tensorflow as tf
tf.random.set_seed(42)
import numpy as np
np.random.seed(42)

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from datetime import datetime
import plotly.offline as pyo
from plotly import subplots
import plotly.graph_objects as go
import statsmodels.api as sm
import statsmodels.tsa.api as smt

from statsmodels.tsa.arima.model import ARIMA
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Model
from sklearn.model_selection import TimeSeriesSplit
from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import LSTM, Dense,Input,concatenate
from tensorflow.keras.layers import *
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam


from baseFunctions import *
from data_helpers import processData6

In [ ]:
data, propDicts, flippedPropDicts = processData6()

In [ ]:
def featureEngineering(data1, frequencies = [12,104,24,52], splits=[1,1,1,1], refTimeSpan=365, feature='day_of_year', oneHotWeekday = False):
    data1 = data.copy()

    # add linear time
    data1['linear_time'] = (data1['date'] - data1['date'].iloc[0]).dt.days +1
    data1['day_of_year'] = data1['date'].dt.day_of_year

    featureNames = []

    for i,f in enumerate(frequencies):
        data1, periodicfeat = addFourierFeature(data1, n_splits = splits[i], frequency=f, feature=feature, referenceTimespan = refTimeSpan)
        featureNames = featureNames + periodicfeat

    data1['weekday'] = data1['date'].dt.weekday
    data1['month'] = data1['date'].dt.month

    if oneHotWeekday:
        one_hot_encoded = pd.get_dummies(data.weekday, prefix='weekday').astype(int)
        data = pd.concat([data, one_hot_encoded], axis=1)
        data = data.drop('weekday', axis = 1)

    return data1, featureNames

In [ ]:
data, timeFeatures = featureEngineering(data)